In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

In [ ]:
# DATA load
X = pd.read_csv('/loan_train_preprocessed.csv')

# backward
X = X[['term', 'initial_list_status', 'int_rate', 
'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 
'inq_last_6mths', 'revol_util', 'recoveries', 
'collection_recovery_fee', 'tot_cur_bal', 
'home_ownershipRENT', 'purposesmall_business', 
'purposewedding', 'earliest_cr_line2000']]

y = pd.read_csv('/loan_train_label.csv')
y = y.drop(['id'], axis=1)

# 새 섹션

# 새 섹션

In [ ]:
# Dividing the data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.20 )
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size = 0.20 )

x_train.shape,y_train.shape,x_val.shape,y_val.shape,x_test.shape,y_test.shape

((10240, 16), (10240, 1), (2560, 16), (2560, 1), (3200, 16), (3200, 1))

In [ ]:
# 수정된 get_clf_eval() 함수 
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
# XGBoost training
evals = [(x_val, y_val)]

xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_wrapper.fit(x_train , y_train,  early_stopping_rounds=100, eval_set=evals, eval_metric="logloss",  verbose=True)

y_preds = xgb_wrapper.predict(x_test)
y_pred_proba = xgb_wrapper.predict_proba(x_test)[:, 1]

[0]	validation_0-logloss:0.6706
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.65153
[2]	validation_0-logloss:0.635886
[3]	validation_0-logloss:0.622483
[4]	validation_0-logloss:0.611157


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[5]	validation_0-logloss:0.601318
[6]	validation_0-logloss:0.592991
[7]	validation_0-logloss:0.585778
[8]	validation_0-logloss:0.57941
[9]	validation_0-logloss:0.573957
[10]	validation_0-logloss:0.569153
[11]	validation_0-logloss:0.565075
[12]	validation_0-logloss:0.561316
[13]	validation_0-logloss:0.558025
[14]	validation_0-logloss:0.555281
[15]	validation_0-logloss:0.552779
[16]	validation_0-logloss:0.550422
[17]	validation_0-logloss:0.548191
[18]	validation_0-logloss:0.546304
[19]	validation_0-logloss:0.544665
[20]	validation_0-logloss:0.543185
[21]	validation_0-logloss:0.541783
[22]	validation_0-logloss:0.540578
[23]	validation_0-logloss:0.539485
[24]	validation_0-logloss:0.538427
[25]	validation_0-logloss:0.537428
[26]	validation_0-logloss:0.536634
[27]	validation_0-logloss:0.535695
[28]	validation_0-logloss:0.535005
[29]	validation_0-logloss:0.534263
[30]	validation_0-logloss:0.533635
[31]	validation_0-logloss:0.533114
[32]	validation_0-logloss:0.532775
[33]	validation_0-logloss:

In [ ]:
get_clf_eval(y_test, y_preds, y_pred_proba)

오차 행렬
[[1159  416]
 [ 540 1085]]
정확도: 0.7013, 정밀도: 0.7229, 재현율: 0.6677,    F1: 0.6942, AUC:0.7879
